# Creación de DataFrames (Tablas Tidy)

In [121]:
# Instalando las paqueterías necesarias.
%pip install pandas openpyxl requests beautifulsoup4
%pip install wget
%pip install gdown
%pip install unidecode


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



Las siguientes librerías nos ayudaran para el proceso de limpieza, creación de tablas tidy y generación del archivo txt.

Se utiliza librería unicode, para poder quitar los acentos de ciertas palabras y usarlo como nombre de archivos a descargar.

In [122]:
# Importación de librerías para usarlas en el proyecto de descarga y generación del txt con la descripción de esta
import pandas as pd
import re
import os
import time
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from logging import exception
from unidecode import unidecode
from datetime import datetime, timedelta

La inicialización de ciertas variables ayudaran a preparar la información inicial de cada una de ellas, además de que puedan ser usadas en cualquier parte del código o proceso, sin en el que perjudique el ámbito o espacio donde estas estén siendo seteadas.

In [123]:
# Inicializando variables que serán usadas más adelante

incidencia_clima_link = 'https://open-meteo.com/en/docs/historical-weather-api'
incidencia_title = ''
incidencia_description = ''

folder_path_data = '../rawdata'
folder_path_tidy_data = '../tidydata'
folder_path_log = '../logs/descarga/'
folder_path_error_log = '../logs/errores/'
csv_filepath = ''

# Creando variable que contendra el listado de cada source descargado.
sources_list = []

# Creando variable que contendra el listado de cada nombre de archivo descargado y que esta siendo enviado a nuestro repositorio
names_files_list = []

texto_list = ''

#### Sección donde se encuentran las funciones generales del proceso

In [124]:
def log_error_write(description, line):
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_error_log):
        os.makedirs(folder_path_error_log)

    date_now = datetime.now()

    # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
    line_error = f"""{date_now.strftime('%Y-%m-%d %H:%M:%S')} {line} - {description}"""
    # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
    with open(f'{folder_path_error_log}/ddf_err{date_now.strftime("%d%m%y")}.txt', 'a', encoding="utf-8") as error_file:
        error_file.write(f"\n{line_error}")

def log_write(description, line):
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_log):
        os.makedirs(folder_path_log)

    date_now = datetime.now()

    # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
    line_write = f"""{date_now.strftime('%Y-%m-%d %H:%M:%S')} {line} - {description}"""
    # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
    with open(f'{folder_path_log}/muncoord{date_now.strftime("%d%m%y")}.txt', 'a', encoding="utf-8") as write_file:
        write_file.write(f"\n{line_write}")


def create_csv_from_DataFrame(df, csv_filename):
    try:

        # Construyendo la ruta completa del archivo
        csv_filepath = os.path.join(folder_path_tidy_data, csv_filename)

        # Guardando el DataFrame como un archivo CSV
        df.to_csv(csv_filepath, index=False, encoding='utf-8')
        print(f"DataFrame (tabla tidy) guardado exitosamente en {csv_filepath}")

    except Exception as e:
        # Imprimiendo un mensaje de error en caso de excepción
        log_error_write(f"Error al guardar el DataFrame (tabla tidy) como CSV: {e}", 'create_csv_from_DataFrame')
        print(f"Error al guardar el DataFrame como CSV: {e}")

#### Obtención de datos de las regiones de sonora que esta relacionado con los municipios

In [400]:
try:
    municipios_regiones = pd.read_csv(f'{folder_path_data}/municipios_regiones.csv')
    municipios_regiones.rename(columns = {'Municipio': 'municipio'}, inplace = True)
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error al leer el archivo CSV: {e}", 'Lectura de las regiones de municipios')
    print(f"Error al leer el archivo CSV: {e}")

      municipio       Region
46  Agua Prieta  Sierra Alta


## Primer DataFrame: Incidencia Delictiva a Nivel Estatal

#### Obtención de datos de incidencia delicitiva a nivel estatal

In [266]:
try:
    # Leemos el archivos csv para obtener el dataframe original
    Incidencia_Delictiva_Estatal_df = pd.read_csv(f'{folder_path_data}/CifrasdeIncidenciaDelictivaEstatal2015agosto2023.csv')
    Incidencia_Delictiva_Estatal_df.columns
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error al leer el archivo CSV: {e}", 'Lectura de CifrasdeIncidenciaDelictivaEstatal2015agosto2023')
    print(f"Error al leer el archivo CSV: {e}")

#### Primer proceso de limpieza y reorganización de la estructura original para crear la tabla tidy correspondiente

In [267]:
mes = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
         'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

try:
  if not Incidencia_Delictiva_Estatal_df.empty:
    Incidencia_Delictiva_Estatal_df = Incidencia_Delictiva_Estatal_df.reset_index()

    # Obtenemos los datos referentes al estado de Sonora y pasamos de un formato wide a uno long
    Incidencia_Delictiva_Estatal_df = Incidencia_Delictiva_Estatal_df[Incidencia_Delictiva_Estatal_df['Entidad'] == 'Sonora']
    Incidencia_Delictiva_Estatal_df = pd.melt(Incidencia_Delictiva_Estatal_df, id_vars = list(Incidencia_Delictiva_Estatal_df.columns[0:8]), value_vars = mes, var_name = 'Mes', value_name = 'Número de delitos')
    Incidencia_Delictiva_Estatal_df = Incidencia_Delictiva_Estatal_df.drop('index', axis = 1)
    Incidencia_Delictiva_Estatal_df = Incidencia_Delictiva_Estatal_df.drop(['Clave_Ent'], axis=1)

    # Checamos que columnas tienen valores faltantes
    for column in Incidencia_Delictiva_Estatal_df.columns:
      print(f'Columna {column}: ' + str(Incidencia_Delictiva_Estatal_df[column].isnull().values.any()))
  else:
      log_error_write(f"DataFrame leido vacio", 'Limpieza Incidencia_Delictiva_Estatal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Incidencia_Delictiva_Estatal_df 1')
    print(f"Error en la limpieza de dataframe: {e}")

Columna Año: False
Columna Entidad: False
Columna Bien jurídico afectado: False
Columna Tipo de delito: False
Columna Subtipo de delito: False
Columna Modalidad: False
Columna Mes: False
Columna Número de delitos: True


#### Segundo proceso de limpieza y reorganización de la estructura de la tabla tidy

In [221]:
try:
  if not Incidencia_Delictiva_Estatal_df.empty:
    # Quitamos observaciones en el futuro o no registradas, esto es, observaciones de los meses de Septiembre, Octubre, NOviembre
    # y Diciembre del año 2023.
    Incidencia_Delictiva_Estatal_df = Incidencia_Delictiva_Estatal_df.drop(Incidencia_Delictiva_Estatal_df[Incidencia_Delictiva_Estatal_df['Mes'].isin(['Septiembre', 'Octubre', 'Noviembre', 'Diciembre']) & (Incidencia_Delictiva_Estatal_df['Año'].astype('string') == '2023')].index)
    #Se hace reenombramiento de las distintas columnas de la tabla tidy para que tenga un formato estandar entre todas las demas tablas tidy
    Incidencia_Delictiva_Estatal_df.rename(columns = {'Año': 'anio'}, inplace = True)
    Incidencia_Delictiva_Estatal_df.rename(columns = {'Mes': 'mes'}, inplace = True)
    Incidencia_Delictiva_Estatal_df.rename(columns = {'Entidad': 'entidad'}, inplace = True)
    Incidencia_Delictiva_Estatal_df.rename(columns = {'Bien jurídico afectado': 'bien_juridico_afectado'}, inplace = True)
    Incidencia_Delictiva_Estatal_df.rename(columns = {'Tipo de delito': 'tipo_delito'}, inplace = True)
    Incidencia_Delictiva_Estatal_df.rename(columns = {'Subtipo de delito': 'subtipo_delito'}, inplace = True)
    Incidencia_Delictiva_Estatal_df.rename(columns = {'Modalidad': 'modalidad'}, inplace = True)     
    Incidencia_Delictiva_Estatal_df.rename(columns = {'Número de delitos': 'numero_delitos'}, inplace = True)

    Incidencia_Delictiva_Estatal_df['numero_delitos'] = Incidencia_Delictiva_Estatal_df['numero_delitos'].round(2).astype(float)
    Incidencia_Delictiva_Estatal_df.insert(1, 'mes', Incidencia_Delictiva_Estatal_df.pop('mes'))
  else:
      log_error_write(f"DataFrame leido vacio", 'Limpieza Incidencia_Delictiva_Estatal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Incidencia_Delictiva_Estatal_df_ 2')
    print(f"Error en la limpieza de dataframe: {e}")

In [278]:
if not Incidencia_Delictiva_Estatal_df.empty:
    print(Incidencia_Delictiva_Estatal_df.shape)

(10584, 8)


In [224]:
try:
  if not Incidencia_Delictiva_Estatal_df.empty:
    # Checamos nuevamente si existen datos faltantes
    for column in Incidencia_Delictiva_Estatal_df.columns:
      print(f'Columna {column}: ' + str(Incidencia_Delictiva_Estatal_df[column].isnull().values.any()))
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Estatal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la lectura de dataframe: {e}", 'Lectura vacios de Incidencia_Delictiva_Estatal_df')
    print(f"Error en la lectura de dataframe: {e}")

Columna anio: False
Columna mes: False
Columna entidad: False
Columna bien_juridico_afectado: False
Columna tipo_delito: False
Columna subtipo_delito: False
Columna modalidad: False
Columna numero_delitos: False


#### Generacion de archivo que contiene los datos de la tabla tidy Incidencia_Delictiva_Estatal

In [225]:
try:
  if not Incidencia_Delictiva_Estatal_df.empty:
    create_csv_from_DataFrame(Incidencia_Delictiva_Estatal_df, f'Incidencia_Delictiva_Estatal.csv')
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Estatal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error creacion de archivo csv: {e}", 'creacion archivo tabla tidy Incidencia_Delictiva_Estatal')
    print(f"Error creacion de archivo csv: {e}")

DataFrame (tabla tidy) guardado exitosamente en ../tidydata\Incidencia_Delictiva_Estatal.csv


In [226]:
Incidencia_Delictiva_Estatal_df.head()

,anio,mes,entidad,bien_juridico_afectado,tipo_delito,subtipo_delito,modalidad,numero_delitos
0,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,23.0
1,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,4.0
2,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,9.0
3,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,0.0
4,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0.0


## Segundo DataFrame: Incidencia delictiva a nivel municipal 2015-2023

#### Obtención de datos de incidencia delicitiva a nivel municipal

In [270]:
try:
    # Leemos el archivos csv para obtener el dataframe original
    Incidencia_Delictiva_Municipal_df = pd.read_csv(f'{folder_path_data}/CifrasdeIncidenciaDelictivaMunicipal2015agosto2023.csv')
    Incidencia_Delictiva_Municipal_df.head()
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error al leer el archivo CSV: {e}", 'Lectura de CifrasdeIncidenciaDelictivaMunicipal2015agosto2023')
    print(f"Error al leer el archivo CSV: {e}")

#### Primer proceso de limpieza y reorganización de la estructura original para crear la tabla tidy correspondiente

In [271]:
try:
  if not Incidencia_Delictiva_Municipal_df.empty:
    # Obtenemos un DataFrame para el estado de Sonora
    Incidencia_Delictiva_Municipal_df2s = Incidencia_Delictiva_Municipal_df[Incidencia_Delictiva_Municipal_df['Entidad'] == 'Sonora']
    Incidencia_Delictiva_Municipal_df2s = Incidencia_Delictiva_Municipal_df2s.reset_index()
    Incidencia_Delictiva_Municipal_df2s = pd.melt(Incidencia_Delictiva_Municipal_df2s, id_vars = list(Incidencia_Delictiva_Municipal_df.columns[0:9]), value_vars = mes, var_name = 'Mes', value_name = 'Número de delitos')

    for column in Incidencia_Delictiva_Municipal_df2s.columns:
      print(f'Columna {column}: ' + str(Incidencia_Delictiva_Municipal_df2s[column].isnull().values.any()))
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Municipal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Incidencia_Delictiva_Municipal_df 1')
    print(f"Error en la limpieza de dataframe: {e}")

Columna Año: False
Columna Clave_Ent: False
Columna Entidad: False
Columna Cve. Municipio: False
Columna Municipio: False
Columna Bien jurídico afectado: False
Columna Tipo de delito: False
Columna Subtipo de delito: False
Columna Modalidad: False
Columna Mes: False
Columna Número de delitos: True


#### Segundo proceso de limpieza y reorganización de la estructura de la tabla tidy

In [272]:
try:
  if not Incidencia_Delictiva_Municipal_df.empty:
    # Quitamos observaciones en el futuro o no registradas, esto es, observaciones de los meses de Septiembre, Octubre, NOviembre
    # y Diciembre del año 2023.
    Incidencia_Delictiva_Municipal_df2s = Incidencia_Delictiva_Municipal_df2s.drop(Incidencia_Delictiva_Municipal_df2s[Incidencia_Delictiva_Municipal_df2s['Mes'].isin(['Septiembre', 'Octubre', 'Noviembre', 'Diciembre']) & (Incidencia_Delictiva_Municipal_df2s['Año'].astype('string') == '2023')].index)

    Incidencia_Delictiva_Municipal_df2s = Incidencia_Delictiva_Municipal_df2s.drop(['Clave_Ent'], axis=1)
    Incidencia_Delictiva_Municipal_df2s = Incidencia_Delictiva_Municipal_df2s.drop(['Cve. Municipio'], axis=1)
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Municipal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Incidencia_Delictiva_Municipal_df 2')
    print(f"Error en la limpieza de dataframe: {e}")

In [276]:
if not Incidencia_Delictiva_Municipal_df.empty:
    print(Incidencia_Delictiva_Municipal_df2s.shape)

(754208, 9)


#### Tercer proceso de limpieza y reorganización de la estructura de la tabla tidy

In [240]:
try:
  if not Incidencia_Delictiva_Municipal_df.empty:
    #print(Incidencia_Delictiva_Municipal_df2s[Incidencia_Delictiva_Municipal_df2s['Municipio'].str.contains('mich', case=False)])

    #Se hace reenombramiento de las distintas columnas de la tabla tidy para que tenga un formato estandar entre todas las demas tablas tidy
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Año': 'anio'}, inplace = True)
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Mes': 'mes'}, inplace = True)
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Entidad': 'entidad'}, inplace = True)
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Municipio': 'municipio'}, inplace = True)
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Bien jurídico afectado': 'bien_juridico_afectado'}, inplace = True)
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Tipo de delito': 'tipo_delito'}, inplace = True)
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Subtipo de delito': 'subtipo_delito'}, inplace = True)
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Modalidad': 'modalidad'}, inplace = True)     
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Número de delitos': 'numero_delitos'}, inplace = True)

    Incidencia_Delictiva_Municipal_df2s['numero_delitos'] = Incidencia_Delictiva_Municipal_df2s['numero_delitos'].round(2).astype(float)


    # Normalizando las columnas en comun (municipio) antes de hacer el merge y agregar al columna region a la tabla tidy
    Incidencia_Delictiva_Municipal_df2s['municipio_normalizada'] = Incidencia_Delictiva_Municipal_df2s['municipio'].apply(unidecode)
    municipios_regiones['municipio_normalizada'] = municipios_regiones['municipio'].apply(unidecode)

    Incidencia_Delictiva_Municipal_df2s = pd.merge(Incidencia_Delictiva_Municipal_df2s, municipios_regiones, on='municipio_normalizada', how='left')

    Incidencia_Delictiva_Municipal_df2s.drop(['municipio_normalizada', 'municipio_y'], axis=1, inplace=True)
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'municipio_x': 'municipio'}, inplace = True)
    Incidencia_Delictiva_Municipal_df2s.rename(columns = {'Region': 'region'}, inplace = True)

    Incidencia_Delictiva_Municipal_df2s.insert(1, 'mes', Incidencia_Delictiva_Municipal_df2s.pop('mes'))
    Incidencia_Delictiva_Municipal_df2s.insert(3, 'region', Incidencia_Delictiva_Municipal_df2s.pop('region'))
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Municipal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Incidencia_Delictiva_Municipal_df 3')
    print(f"Error en la limpieza de dataframe: {e}")


In [241]:
try:
  if not Incidencia_Delictiva_Municipal_df.empty:
    # Verificamos si existen otros datos faltantes
    for column in Incidencia_Delictiva_Municipal_df2s.columns:
      print(f'Columna {column}: ' + str(Incidencia_Delictiva_Municipal_df2s[column].isnull().values.any()))

  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Municipal_df')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la lectura de dataframe: {e}", 'Lectura vacios de Incidencia_Delictiva_Municipal_df')
    print(f"Error en la lectura de dataframe: {e}")
  

Columna anio: False
Columna mes: False
Columna entidad: False
Columna region: True
Columna municipio: False
Columna bien_juridico_afectado: False
Columna tipo_delito: False
Columna subtipo_delito: False
Columna modalidad: False
Columna numero_delitos: False


#### Generacion de archivo que contiene los datos de la tabla tidy Incidencia_Delictiva_Municipal

In [242]:
try:
  if not Incidencia_Delictiva_Municipal_df.empty:
    create_csv_from_DataFrame(Incidencia_Delictiva_Municipal_df2s, f'Incidencia_Delictiva_Municipal.csv')
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Municipal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error creacion de archivo csv: {e}", 'creacion archivo tabla tidy Incidencia_Delictiva_Municipal')
    print(f"Error creacion de archivo csv: {e}")

DataFrame (tabla tidy) guardado exitosamente en ../tidydata\Incidencia_Delictiva_Municipal.csv


In [243]:
Incidencia_Delictiva_Municipal_df2s.head()

,anio,mes,entidad,region,municipio,bien_juridico_afectado,tipo_delito,subtipo_delito,modalidad,numero_delitos
0,2015,Enero,Sonora,Río Sonora,Aconchi,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,0.0
1,2015,Enero,Sonora,Río Sonora,Aconchi,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,0.0
2,2015,Enero,Sonora,Río Sonora,Aconchi,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0.0
3,2015,Enero,Sonora,Río Sonora,Aconchi,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,0.0
4,2015,Enero,Sonora,Río Sonora,Aconchi,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,0.0


## Tercer DataFrame: Incidencia delictiva federal 2012-2023

#### Obtención de datos de incidencia delicitiva federal

In [279]:
try:
    # Leemos el archivo csv
    Incidencia_Delictiva_Federal_df = pd.read_csv(f'{folder_path_data}/CifrasdeIncidenciaDelictivaFederal2012agosto2023.csv')
    Incidencia_Delictiva_Federal_df3s = Incidencia_Delictiva_Federal_df[Incidencia_Delictiva_Federal_df['ENTIDAD'] == 'SONORA']
    Incidencia_Delictiva_Federal_df3s.head()
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error al leer el archivo CSV: {e}", 'Lectura de CifrasdeIncidenciaDelictivaFederal2012agosto2023')
    print(f"Error al leer el archivo CSV: {e}")

#### Primer proceso de limpieza y reorganización de la estructura original para crear la tabla tidy correspondiente

In [280]:
try:
    if not Incidencia_Delictiva_Federal_df.empty:
        Mes = list(np.char.upper(np.array(mes)))

        Incidencia_Delictiva_Federal_df3s = Incidencia_Delictiva_Federal_df3s.reset_index()
        Incidencia_Delictiva_Federal_df3s = pd.melt(Incidencia_Delictiva_Federal_df3s, id_vars = list(Incidencia_Delictiva_Federal_df.columns[0:6]),
               value_vars = Mes, var_name = 'MES', value_name = 'NÚMERO DE DELITOS')
    else:
        log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Federal_df')
        print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Incidencia_Delictiva_Federal_df 1')
    print(f"Error en la limpieza de dataframe: {e}")

In [281]:
if not Incidencia_Delictiva_Federal_df.empty:
    Incidencia_Delictiva_Federal_df3s.head()

#### Segundo proceso de limpieza y reorganización de la estructura de la tabla tidy

In [282]:
try:
    if not Incidencia_Delictiva_Federal_df.empty:
        # Quitamos observaciones en el futuro o no registradas, esto es, observaciones de los meses de Septiembre, Octubre, NOviembre
        # y Diciembre del año 2023.
        Incidencia_Delictiva_Federal_df3s = Incidencia_Delictiva_Federal_df3s.drop(Incidencia_Delictiva_Federal_df3s[Incidencia_Delictiva_Federal_df3s['MES'].isin(['SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']) & (Incidencia_Delictiva_Federal_df3s['AÑO'].astype('string') == '2023')].index)

        Incidencia_Delictiva_Federal_df3s = Incidencia_Delictiva_Federal_df3s.drop(['INEGI'], axis=1)
    else:
        log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Federal_df')
        print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Incidencia_Delictiva_Federal_df 2')
    print(f"Error en la limpieza de dataframe: {e}")

In [283]:
if not Incidencia_Delictiva_Federal_df.empty:
    print(Incidencia_Delictiva_Federal_df3s.shape)

(4032, 7)


#### Tercer proceso de limpieza y reorganización de la estructura de la tabla tidy

In [284]:
try:
    if not Incidencia_Delictiva_Federal_df.empty:
        #Se hace reenombramiento de las distintas columnas de la tabla tidy para que tenga un formato estandar entre todas las demas tablas tidy
        Incidencia_Delictiva_Federal_df3s.rename(columns = {'AÑO': 'anio'}, inplace = True)
        Incidencia_Delictiva_Federal_df3s.rename(columns = {'MES': 'mes'}, inplace = True)
        Incidencia_Delictiva_Federal_df3s.rename(columns = {'ENTIDAD': 'entidad'}, inplace = True)
        Incidencia_Delictiva_Federal_df3s.rename(columns = {'LEY': 'ley'}, inplace = True)
        Incidencia_Delictiva_Federal_df3s.rename(columns = {'CONCEPTO': 'concepto'}, inplace = True)
        Incidencia_Delictiva_Federal_df3s.rename(columns = {'TIPO': 'tipo'}, inplace = True)     
        Incidencia_Delictiva_Federal_df3s.rename(columns = {'NÚMERO DE DELITOS': 'numero_delitos'}, inplace = True)

        Incidencia_Delictiva_Federal_df3s['numero_delitos'] = Incidencia_Delictiva_Federal_df3s['numero_delitos'].round(2).astype(float)

        for column in Incidencia_Delictiva_Federal_df3s.columns:
            if Incidencia_Delictiva_Federal_df3s[column].dtype == 'object':
                Incidencia_Delictiva_Federal_df3s[column] = Incidencia_Delictiva_Federal_df3s[column].apply(lambda x: x.lower().title())

        Incidencia_Delictiva_Federal_df3s.insert(1, 'mes', Incidencia_Delictiva_Federal_df3s.pop('mes'))
    else:
        log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Federal_df')
        print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Incidencia_Delictiva_Federal_df 3')
    print(f"Error en la limpieza de dataframe: {e}")

In [285]:
try:
  if not Incidencia_Delictiva_Federal_df.empty:
    # Verificamos si hay datos faltantes
    for column in Incidencia_Delictiva_Federal_df3s.columns:
      print(f'Columna {column}: ' + str(Incidencia_Delictiva_Federal_df3s[column].isnull().values.any()))
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Federal_df')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la lectura de dataframe: {e}", 'Lectura vacios de Incidencia_Delictiva_Federal_df')
    print(f"Error en la lectura de dataframe: {e}")

Columna anio: False
Columna mes: False
Columna entidad: False
Columna ley: False
Columna concepto: False
Columna tipo: False
Columna numero_delitos: False


#### Generacion de archivo que contiene los datos de la tabla tidy Incidencia_Delictiva_Federal

In [286]:
try:
  if not Incidencia_Delictiva_Federal_df.empty:
    create_csv_from_DataFrame(Incidencia_Delictiva_Federal_df3s, f'Incidencia_Delictiva_Federal.csv')
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Incidencia_Delictiva_Federal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error creacion de archivo csv: {e}", 'creacion archivo tabla tidy Incidencia_Delictiva_Federal')
    print(f"Error creacion de archivo csv: {e}")

DataFrame (tabla tidy) guardado exitosamente en ../tidydata\Incidencia_Delictiva_Federal.csv


In [287]:
Incidencia_Delictiva_Federal_df3s.head()

,anio,mes,entidad,ley,concepto,tipo,numero_delitos
0,2012,Enero,Sonora,Codigo Penal Federal,Contra La Salud,Produccion,10.0
1,2012,Enero,Sonora,Codigo Penal Federal,Contra La Salud,Transporte,21.0
2,2012,Enero,Sonora,Codigo Penal Federal,Contra La Salud,Trafico,0.0
3,2012,Enero,Sonora,Codigo Penal Federal,Contra La Salud,Comercio,45.0
4,2012,Enero,Sonora,Codigo Penal Federal,Contra La Salud,Suministro,1.0


## Cuarto DataFrame: Victimas del fuero común 2015-2023

#### Obtención de datos de las victimas del fuero común

In [288]:
try:
    ## Leemos el archivo csv y obtenemos los datos de Sonora
    Victimas_Fuero_Comun_Estatal_df = pd.read_csv(f'{folder_path_data}/CifrasdeVictimasdelFueroComun2015agosto2023.csv')
    Victimas_Fuero_Comun_Estatal_df4s = Victimas_Fuero_Comun_Estatal_df[Victimas_Fuero_Comun_Estatal_df['Entidad'] == 'Sonora']
    Victimas_Fuero_Comun_Estatal_df4s.head()
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error al leer el archivo CSV: {e}", 'Lectura de CifrasdeVictimasdelFueroComun2015agosto2023')
    print(f"Error al leer el archivo CSV: {e}")

In [289]:
try:
  if not Victimas_Fuero_Comun_Estatal_df.empty:
    for column in Victimas_Fuero_Comun_Estatal_df4s.columns:
      print(f'Columna {column}: ' + str(Victimas_Fuero_Comun_Estatal_df4s[column].isnull().values.any()))
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Victimas_Fuero_Comun_Estatal_df')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la lectura de dataframe: {e}", 'Lectura vacios de Victimas_Fuero_Comun_Estatal_df')
    print(f"Error en la lectura de dataframe: {e}")

Columna Año: False
Columna Clave_Ent: False
Columna Entidad: False
Columna Bien jurídico afectado: False
Columna Tipo de delito: False
Columna Subtipo de delito: False
Columna Modalidad: False
Columna Sexo: False
Columna Rango de edad: False
Columna Enero: False
Columna Febrero: False
Columna Marzo: False
Columna Abril: False
Columna Mayo: False
Columna Junio: False
Columna Julio: False
Columna Agosto: False
Columna Septiembre: True
Columna Octubre: True
Columna Noviembre: True
Columna Diciembre: True


#### Primer proceso de limpieza y reorganización de la estructura original para crear la tabla tidy correspondiente

In [290]:
try:
  if not Victimas_Fuero_Comun_Estatal_df.empty:
    # Pasamos de formato wide a long
    Victimas_Fuero_Comun_Estatal_df4s = Victimas_Fuero_Comun_Estatal_df4s.reset_index()
    Victimas_Fuero_Comun_Estatal_df4s = pd.melt(Victimas_Fuero_Comun_Estatal_df4s, id_vars = list(Victimas_Fuero_Comun_Estatal_df.columns[0:9]),
               value_vars = mes, var_name = 'Mes', value_name = 'Número de delitos')
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Victimas_Fuero_Comun_Estatal_df')
    print(f"DataFrame leido vacio")
except Exception as e:
  # Imprimiendo un mensaje de error en caso de excepción
  log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Victimas_Fuero_Comun_Estatal_df 1')
  print(f"Error en la limpieza de dataframe: {e}")

#### Segundo proceso de limpieza y reorganización de la estructura de la tabla tidy

In [291]:
try:
  if not Victimas_Fuero_Comun_Estatal_df.empty:
    # Quitamos observaciones en el futuro o no registradas, esto es, observaciones de los meses de Septiembre, Octubre, NOviembre
    # y Diciembre del año 2023.
    Victimas_Fuero_Comun_Estatal_df4s = Victimas_Fuero_Comun_Estatal_df4s.drop(Victimas_Fuero_Comun_Estatal_df4s[Victimas_Fuero_Comun_Estatal_df4s['Mes'].isin(['Septiembre', 'Octubre', 'Noviembre', 'Diciembre']) & (Victimas_Fuero_Comun_Estatal_df4s['Año'].astype('string') == '2023')].index)

    Victimas_Fuero_Comun_Estatal_df4s = Victimas_Fuero_Comun_Estatal_df4s.drop(['Clave_Ent'], axis=1)
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Victimas_Fuero_Comun_Estatal_df')
    print(f"DataFrame leido vacio")
except Exception as e:
  # Imprimiendo un mensaje de error en caso de excepción
  log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Victimas_Fuero_Comun_Estatal_df 2')
  print(f"Error en la limpieza de dataframe: {e}")

In [292]:
if not Victimas_Fuero_Comun_Estatal_df.empty:
    print(Victimas_Fuero_Comun_Estatal_df4s.shape)

(23920, 10)


#### Tercer proceso de limpieza y reorganización de la estructura de la tabla tidy

In [293]:
try:
  if not Victimas_Fuero_Comun_Estatal_df.empty:
    #Se hace reenombramiento de las distintas columnas de la tabla tidy para que tenga un formato estandar entre todas las demas tablas tidy
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Año': 'anio'}, inplace = True)
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Mes': 'mes'}, inplace = True)
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Entidad': 'entidad'}, inplace = True)
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Bien jurídico afectado': 'bien_juridico_afectado'}, inplace = True)
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Tipo de delito': 'tipo_delito'}, inplace = True)
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Subtipo de delito': 'subtipo_delito'}, inplace = True)
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Modalidad': 'modalidad'}, inplace = True)
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Sexo': 'sexo'}, inplace = True)
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Rango de edad': 'rango_edad'}, inplace = True)     
    Victimas_Fuero_Comun_Estatal_df4s.rename(columns = {'Número de delitos': 'numero_delitos'}, inplace = True)

    Victimas_Fuero_Comun_Estatal_df4s['numero_delitos'] = Victimas_Fuero_Comun_Estatal_df4s['numero_delitos'].round(2).astype(float)

    Victimas_Fuero_Comun_Estatal_df4s.insert(1, 'mes', Victimas_Fuero_Comun_Estatal_df4s.pop('mes'))
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Victimas_Fuero_Comun_Estatal_df')
    print(f"DataFrame leido vacio")
except Exception as e:
  # Imprimiendo un mensaje de error en caso de excepción
  log_error_write(f"Error en la limpieza de dataframe: {e}", 'Limpieza Victimas_Fuero_Comun_Estatal_df 3')
  print(f"Error en la limpieza de dataframe: {e}")

In [294]:
try:
  if not Victimas_Fuero_Comun_Estatal_df.empty:
    # Verificamos si existen otros datos faltantes
    for column in Victimas_Fuero_Comun_Estatal_df4s.columns:
      print(f'Columna {column}: ' + str(Victimas_Fuero_Comun_Estatal_df4s[column].isnull().values.any()))
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Victimas_Fuero_Comun_Estatal_df')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la lectura de dataframe: {e}", 'Lectura vacios de Victimas_Fuero_Comun_Estatal_df')
    print(f"Error en la lectura de dataframe: {e}")

Columna anio: False
Columna mes: False
Columna entidad: False
Columna bien_juridico_afectado: False
Columna tipo_delito: False
Columna subtipo_delito: False
Columna modalidad: False
Columna sexo: False
Columna rango_edad: False
Columna numero_delitos: False


#### Generacion de archivo que contiene los datos de la tabla tidy Victimas_Fuero_Comun_Estatal

In [295]:
try:
  if not Victimas_Fuero_Comun_Estatal_df.empty:
    create_csv_from_DataFrame(Victimas_Fuero_Comun_Estatal_df4s, f'Victimas_Fuero_Comun_Estatal.csv')
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de Victimas_Fuero_Comun_Estatal_df')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error creacion de archivo csv: {e}", 'creacion archivo tabla tidy Victimas_Fuero_Comun_Estatal')
    print(f"Error creacion de archivo csv: {e}")

DataFrame (tabla tidy) guardado exitosamente en ../tidydata\Victimas_Fuero_Comun_Estatal.csv


In [298]:
Victimas_Fuero_Comun_Estatal_df4s.head()
#Incidencia_Delictiva_Estatal_df.info()

,anio,mes,entidad,bien_juridico_afectado,tipo_delito,subtipo_delito,modalidad,sexo,rango_edad,numero_delitos
0,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,Mujer,Menores de edad (0-17),0.0
1,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,Mujer,Menores de edad (0-17),0.0
2,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,Mujer,Menores de edad (0-17),0.0
3,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio doloso,No especificado,Mujer,Menores de edad (0-17),0.0
4,2015,Enero,Sonora,La vida y la Integridad corporal,Homicidio,Homicidio culposo,Con arma de fuego,Mujer,Menores de edad (0-17),0.0


## Datos tidy sobre el clima

#### Obtención de datos meteorológicos y climatológicos

In [479]:
# Leemos archivo csv de los datos raw de clima
try:
   dfclima = pd.read_csv(f'{folder_path_data}/DatosClimaMunicipal.csv')
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error al leer el archivo CSV: {e}", 'Lectura de DatosClimaMunicipal')
    print(f"Error al leer el archivo CSV: {e}")

#### Primer proceso de limpieza y reorganización de la estructura original para crear la tabla tidy correspondiente

In [480]:
try:
  if not dfclima.empty:
    #Renombramos previamente algunas columnas para facilitar el manejo del dataframe
    dfclima.rename(columns = {'time':'Dia', 'temperature_2m_max': 'Temp_max',
                            'temperature_2m_min':'Temp_min', 'sunrise':'Salida_sol',
                            'sunset':'Puesta_sol', 'rain_sum':'Lluvia', 
                            'precipitation_hours': 'Tiempo_precipitacion',
                            'temperature_2m_mean': 'Temperatura_promedio',
                            'precipitation_sum':'Precipitacion_total',
                            'municipio': 'Municipio'}, inplace = True)
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'renombramiento del dataframe dfclima')
    print(f"Error renombramiento: {e}")

In [481]:
# Asignamos a fecha_final la última fecha a considerar en nuestros datos, transformamos la columna con la fecha
# a tipo de dato datetime y removemos todos aquellos registros que estén despues de fecha_final
try:
  if not dfclima.empty:
    fecha_final = np.datetime64('2023-09-29')
    dfclima['Dia'] = pd.to_datetime(dfclima['Dia'])
    dfclima = dfclima.drop(dfclima[dfclima['Dia'] >= fecha_final].index)

  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza dfclima 1')
    print(f"Error renombramiento: {e}")

In [482]:
# Separamos la columna con la fecha en año y mes, y las anexamos al dataframe
try:
  if not dfclima.empty:
    columna_anio = dfclima['Dia'].dt.year
    columna_mes = dfclima['Dia'].dt.month

    dfclima.insert(0, 'anio', columna_anio)
    dfclima.insert(1, 'mes', columna_mes)

    dfclima['Dia'] = dfclima['Dia'].dt.day

    # Transformamos las columnas que corresponden a la salida y puesta de sol a tipo de dato datetime
    dfclima['Salida_sol'] = pd.to_datetime(dfclima['Salida_sol'])
    dfclima['Puesta_sol'] = pd.to_datetime(dfclima['Puesta_sol'])
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza dfclima 2')
    print(f"Error renombramiento: {e}")

In [483]:
if not dfclima.empty:
    print(dfclima.head())

   anio  mes  Dia  Temp_max  Temp_min  Temperatura_promedio  \
0  2015    1    1      14.6       8.6                  11.0   
1  2015    1    2      12.1       3.4                   7.5   
2  2015    1    3      16.3      -0.6                   7.4   
3  2015    1    4      22.7       3.5                  11.5   
4  2015    1    5      24.3       5.8                  13.6   

           Salida_sol          Puesta_sol  Precipitacion_total  Lluvia  \
0 2015-01-01 07:16:00 2015-01-01 17:32:00                  0.1     0.1   
1 2015-01-02 07:16:00 2015-01-02 17:33:00                  0.0     0.0   
2 2015-01-03 07:16:00 2015-01-03 17:33:00                  0.0     0.0   
3 2015-01-04 07:16:00 2015-01-04 17:34:00                  0.0     0.0   
4 2015-01-05 07:17:00 2015-01-05 17:35:00                  0.0     0.0   

   Tiempo_precipitacion Municipio  
0                   1.0   Aconchi  
1                   0.0   Aconchi  
2                   0.0   Aconchi  
3                   0.0   Aconch

In [484]:
# Verificamos si existen datos faltantes
try:
  if not dfclima.empty:
    for column in dfclima.columns:
      print(f'Columna {column}: ' + str(dfclima[column].isnull().values.any()))
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la lectura de dataframe: {e}", 'Lectura vacios de dfclima')
    print(f"Error en la lectura de dataframe: {e}")

Columna anio: False
Columna mes: False
Columna Dia: False
Columna Temp_max: False
Columna Temp_min: False
Columna Temperatura_promedio: False
Columna Salida_sol: False
Columna Puesta_sol: False
Columna Precipitacion_total: False
Columna Lluvia: False
Columna Tiempo_precipitacion: False
Columna Municipio: False


In [485]:
try:
  if not dfclima.empty:
    # Obtenemos la salida y puesta de sol en minutos, y asignamos estos datos a dos nuevas 
    # columnas: sal_sol y puesta_sol
    dfclima['salida_sol'] = dfclima['Salida_sol'].map(lambda x: (x.hour)*60 + x.minute)
    dfclima['puesta_sol'] = dfclima['Puesta_sol'].map(lambda x: (x.hour)*60 + x.minute)

    # Calculamos la cantidad de tiempo en minutos en los que hay luz restando las dos columnas anteriores
    dfclima['minutos_luz'] = dfclima['puesta_sol'] - dfclima['salida_sol']
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza dfclima 2')
    print(f"Error renombramiento: {e}")

## Meteorologia_Climatologia_Municipal

#### Segundo proceso de limpieza y reorganización de la estructura original para crear la tabla tidy correspondiente los datos meteorológicos y climatológicos a nivel municipal

In [486]:
try:
  if not dfclima.empty:
    # Agrupamos el dataframe dfclima por año, mes y municipio, y utilizamos distintas funciones de agregación
    dfclima_mes = dfclima.groupby(['anio','mes','Municipio'], as_index = False).agg({'Temperatura_promedio': ['mean'],
                                                    'Temp_max': ['mean'], 'Temp_min': ['mean'],
                                                    'Tiempo_precipitacion': 'sum', 'Lluvia':'sum',
                                                    'Precipitacion_total': 'sum', 'minutos_luz' : 'mean',
                                                    'salida_sol': 'mean', 'puesta_sol': 'mean'})
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Agrupamiento de datafrane dfclima (municipal)')
    print(f"Error renombramiento: {e}")

In [487]:
# Creamos un diccionario que devuelve el mes dado su valor ordinal en el calendario
dict_mes = {
    1 : 'Enero',
    2 : 'Febrero',
    3 : 'Marzo',
    4 : 'Abril',
    5 : 'Mayo',
    6 : 'Junio',
    7 : 'Julio',
    8 : 'Agosto',
    9 : 'Septiembre',
    10 : 'Octubre',
    11 : 'Noviembre',
    12 : 'Diciembre'
}

In [488]:
try:
  if not dfclima.empty:
    # Eliminamos el multiíndice que se obtiene al momento de utilizar la función groupby
    dfclima_mes.columns = dfclima_mes.columns.to_flat_index()
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza dfclima 1 (municipal)')
    print(f"Error renombramiento: {e}")

In [489]:
dfclima_mes.columns

Index([                    ('anio', ''),                      ('mes', ''),
                      ('Municipio', ''), ('Temperatura_promedio', 'mean'),
                   ('Temp_max', 'mean'),             ('Temp_min', 'mean'),
        ('Tiempo_precipitacion', 'sum'),                ('Lluvia', 'sum'),
         ('Precipitacion_total', 'sum'),          ('minutos_luz', 'mean'),
                 ('salida_sol', 'mean'),           ('puesta_sol', 'mean')],
      dtype='object')

In [490]:
try:
  if not dfclima.empty:
    # Creamos columna Mes con los meses correspondientes con nombre, no valor ordinal, así como la colummna Entidad
    dfclima_mes['Mes'] = dfclima_mes[('mes', '')].map(lambda x : dict_mes[x])
    dfclima_mes['entidad'] = 'Sonora'

    # Redondeamos el promedio de salida y puesta de sol y las ponemos en formato HH:MM
    dfclima_mes[('salida_sol', 'mean')] = dfclima_mes[('salida_sol', 'mean')].round()
    dfclima_mes[('puesta_sol', 'mean')] = dfclima_mes[('puesta_sol', 'mean')].round()
    dfclima_mes['salida_sol'] = dfclima_mes[('salida_sol', 'mean')].map(lambda x: 
                                                                    str(int(x // 60)) + ':' + "{:02d}".format(int(x % 60)))
    dfclima_mes['puesta_sol'] = dfclima_mes[('puesta_sol', 'mean')].map(lambda x: 
                                                                    str(int(x // 60)) + ':' + "{:02d}".format(int(x % 60)))
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza dfclima 2 (municipal)')
    print(f"Error renombramiento: {e}")

In [491]:
if not dfclima.empty:
    print(dfclima_mes.columns)

Index([                    ('anio', ''),                      ('mes', ''),
                      ('Municipio', ''), ('Temperatura_promedio', 'mean'),
                   ('Temp_max', 'mean'),             ('Temp_min', 'mean'),
        ('Tiempo_precipitacion', 'sum'),                ('Lluvia', 'sum'),
         ('Precipitacion_total', 'sum'),          ('minutos_luz', 'mean'),
                 ('salida_sol', 'mean'),           ('puesta_sol', 'mean'),
                                  'Mes',                        'entidad',
                           'salida_sol',                     'puesta_sol'],
      dtype='object')


In [492]:
if not dfclima.empty:
    # Quitamos columna mes antigua
    dfclima_mes = dfclima_mes.drop([('mes', '')], axis = 1)

In [493]:
try:
  if not dfclima_mes.empty:
    # Renombramos las nuevas columnas
    dfclima_mes.rename(columns = {('anio', ''):'anio', ('Municipio', ''):'municipio',
                            ('Temp_max', 'mean'):'temperatura_maxima',
                            ('Temp_min', 'mean'):'temperatura_minima', ('Temperatura_promedio', 'mean'):'temperatura_promedio', 
                            ('Precipitacion_total', 'sum'): 'precipitacion_total',
                            ('Lluvia', 'sum'): 'lluvia_total',
                            ('Tiempo_precipitacion', 'sum'):'tiempo_precipitacion',
                            ('minutos_luz', 'mean'):'minutos_luz', 'Mes': 'mes'}, inplace = True)
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima_mes')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'renombramiento del dataframe dfclima_mes')
    print(f"Error renombramiento: {e}")

In [494]:
if not dfclima_mes.empty:
    #Quitamos columnas ('salida_sol', 'mean') y ('puesta_sol', 'mean') antiguas y se obtiene 
    # el dataframe final por municipio
    df_clima_mun = dfclima_mes.drop([('salida_sol', 'mean'), ('puesta_sol', 'mean')], axis = 1)

In [495]:
if not dfclima_mes.empty:
    print(df_clima_mun.head(3))

   anio    municipio  temperatura_promedio  temperatura_maxima  \
0  2015      Aconchi             15.054839           22.374194   
1  2015  Agua-Prieta             10.961290           18.087097   
2  2015       Alamos             18.258065           25.367742   

   temperatura_minima  tiempo_precipitacion  lluvia_total  \
0            8.912903                  56.0          81.7   
1            4.622581                  68.0          60.7   
2           12.309677                  47.0          45.0   

   precipitacion_total  minutos_luz    mes entidad salida_sol puesta_sol  
0                 81.7   628.645161  Enero  Sonora       7:15      17:44  
1                 60.7   624.064516  Enero  Sonora       7:14      17:38  
2                 45.0   639.419355  Enero  Sonora       7:05      17:44  


In [496]:
if not df_clima_mun.empty:
    # Ordenamos las columnas 
    df_clima_mun = df_clima_mun[['anio', 'mes', 'entidad',
                                'municipio', 'temperatura_maxima',
                                'temperatura_minima', 'temperatura_promedio', 'salida_sol',
                                'puesta_sol', 'precipitacion_total', 'lluvia_total', 'tiempo_precipitacion', 'minutos_luz']]

In [497]:
try:
  if not df_clima_mun.empty:
    # Redondeamos con dos decimales las columnas mostradas
    df_clima_mun[['temperatura_maxima', 'temperatura_minima']] = df_clima_mun[['temperatura_maxima', 'temperatura_minima']].round(decimals = 2)
    df_clima_mun[['temperatura_promedio', 'minutos_luz']] = df_clima_mun[['temperatura_promedio', 'minutos_luz']].round(decimals = 2)
    df_clima_mun[['precipitacion_total', 'lluvia_total']] = df_clima_mun[['precipitacion_total', 'lluvia_total']].round(decimals = 2)
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de df_clima_mun')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza df_clima_mun 1 (municipal)')
    print(f"Error renombramiento: {e}")

In [498]:
try:
  if not dfclima.empty:
    # Reemplazar caracteres especiales por espacios en la columna 'columna'
    df_clima_mun['municipio'] = df_clima_mun['municipio'].replace('[^a-zA-Z0-9 ]', ' ', regex=True)
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza df_clima_mun 2 (municipal)')
    print(f"Error renombramiento: {e}")

In [499]:
try:
  if not dfclima.empty:
    # Normalizando las columnas en comun (municipio) antes de hacer el merge y agregar al columna region a la tabla tidy
    df_clima_mun['municipio_normalizada'] = df_clima_mun['municipio'].apply(unidecode)
    municipios_regiones['municipio_normalizada'] = municipios_regiones['municipio'].apply(unidecode)

    df_clima_mun = pd.merge(df_clima_mun, municipios_regiones, on='municipio_normalizada', how='left')

    df_clima_mun.drop(['municipio_normalizada', 'municipio_y'], axis=1, inplace=True)
    df_clima_mun.rename(columns = {'municipio_x': 'municipio'}, inplace = True)
    df_clima_mun.rename(columns = {'Region': 'region'}, inplace = True)

    df_clima_mun.insert(3, 'region', df_clima_mun.pop('region'))
    df_clima_mun.insert(10, 'minutos_luz', df_clima_mun.pop('minutos_luz'))
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza df_clima_mun 3 (municipal)')
    print(f"Error renombramiento: {e}")

In [500]:
try:
  if not dfclima.empty:
    # Verificamos si existen otros datos faltantes
    for column in df_clima_mun.columns:
      print(f'Columna {column}: ' + str(df_clima_mun[column].isnull().values.any()))
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error en la lectura de dataframe: {e}", 'Lectura vacios de df_clima_mun')
    print(f"Error en la lectura de dataframe: {e}")

Columna anio: False
Columna mes: False
Columna entidad: False
Columna region: False
Columna municipio: False
Columna temperatura_maxima: False
Columna temperatura_minima: False
Columna temperatura_promedio: False
Columna salida_sol: False
Columna puesta_sol: False
Columna minutos_luz: False
Columna precipitacion_total: False
Columna lluvia_total: False
Columna tiempo_precipitacion: False


In [501]:
if not dfclima.empty:
    print(df_clima_mun.head(4))

   anio    mes entidad       region    municipio  temperatura_maxima  \
0  2015  Enero  Sonora   Río Sonora      Aconchi               22.37   
1  2015  Enero  Sonora  Sierra Alta  Agua Prieta               18.09   
2  2015  Enero  Sonora          Sur       Alamos               25.37   
3  2015  Enero  Sonora     Noraeste        Altar               22.10   

   temperatura_minima  temperatura_promedio salida_sol puesta_sol  \
0                8.91                 15.05       7:15      17:44   
1                4.62                 10.96       7:14      17:38   
2               12.31                 18.26       7:05      17:44   
3                7.84                 14.94       7:24      17:49   

   minutos_luz  precipitacion_total  lluvia_total  tiempo_precipitacion  
0       628.65                 81.7          81.7                  56.0  
1       624.06                 60.7          60.7                  68.0  
2       639.42                 45.0          45.0                  47.0

#### Generacion de archivo que contiene los datos de la tabla tidy Meteorologia_Climatologia_Municipal

In [502]:
# Pasamos este dataframe a archivo csv
try:
  if not dfclima.empty:
    create_csv_from_DataFrame(df_clima_mun, f'Meteorologia_Climatologia_Municipal.csv')
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error creacion de archivo csv: {e}", 'creacion archivo tabla tidy Meteorologia_Climatologia_Municipal')
    print(f"Error creacion de archivo csv: {e}")

DataFrame (tabla tidy) guardado exitosamente en ../tidydata\Meteorologia_Climatologia_Municipal.csv


## Meteorologia_Climatologia_Estatal

#### Tercer proceso de limpieza y reorganización de la estructura original para crear la tabla tidy correspondiente los datos meteorológicos y climatológicos a nivel estatal

In [503]:
try:
  if not dfclima.empty:
    # Agrupamos el dataframe dfclima con año y mes, y utilizamos distintas funciones de agregación
    dfclima_est = dfclima.groupby(['anio','mes'], as_index = False).agg({'Temperatura_promedio': ['mean'],
                                                    'Temp_max': ['mean'], 'Temp_min': ['mean'],
                                                    'Tiempo_precipitacion': 'sum', 'Lluvia':'sum',
                                                    'Precipitacion_total': 'sum', 'minutos_luz' : 'mean',
                                                    'salida_sol': 'mean', 'puesta_sol': 'mean'})
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Agrupamiento de datafrane dfclima_est (estatal)')
    print(f"Error renombramiento: {e}")

In [504]:
try:
  if not dfclima.empty:
    # Quitamos multiíndice obtenido por la función groupby
    dfclima_est.columns = dfclima_est.columns.to_flat_index()

    # Creamos columna Mes con nombres y la columna entidad
    dfclima_est['Mes'] = dfclima_est[('mes', '')].map(lambda x : dict_mes[x])
    dfclima_est['entidad'] = 'Sonora'

    # Redondeamos el promedio y la salida de la puesta de sol, y creamos salida_sol y puesta_sol con formato HH:MM
    dfclima_est[('salida_sol', 'mean')] = dfclima_est[('salida_sol', 'mean')].round()
    dfclima_est[('puesta_sol', 'mean')] = dfclima_est[('puesta_sol', 'mean')].round()
    dfclima_est['salida_sol'] = dfclima_est[('salida_sol', 'mean')].map(lambda x: 
                                                                    str(int(x // 60)) + ':' + "{:02d}".format(int(x % 60)))
    dfclima_est['puesta_sol'] = dfclima_est[('puesta_sol', 'mean')].map(lambda x: 
                                                                    str(int(x // 60)) + ':' + "{:02d}".format(int(x % 60)))
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza dfclima_est 1 (estatal)')
    print(f"Error renombramiento: {e}")

In [505]:
if not dfclima.empty:
    print(dfclima_est.columns)

Index([                    ('anio', ''),                      ('mes', ''),
       ('Temperatura_promedio', 'mean'),             ('Temp_max', 'mean'),
                   ('Temp_min', 'mean'),  ('Tiempo_precipitacion', 'sum'),
                      ('Lluvia', 'sum'),   ('Precipitacion_total', 'sum'),
                ('minutos_luz', 'mean'),           ('salida_sol', 'mean'),
                 ('puesta_sol', 'mean'),                            'Mes',
                              'entidad',                     'salida_sol',
                           'puesta_sol'],
      dtype='object')


In [506]:
try:
  if not dfclima.empty:
    # Eliminamos columna mes antigua
    dfclima_est = dfclima_est.drop([('mes', '')], axis = 1)
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza dfclima_est 2 (estatal)')
    print(f"Error renombramiento: {e}")

In [507]:
try:
  if not dfclima.empty:
    # Renombramos columnas
    dfclima_est.rename(columns = {('anio', ''):'anio',
                            ('Temp_max', 'mean'):'temperatura_maxima',
                            ('Temp_min', 'mean'):'temperatura_minima', ('Temperatura_promedio', 'mean'):'temperatura_promedio', 
                            ('Precipitacion_total', 'sum'): 'precipitacion_total',
                            ('Lluvia', 'sum'): 'lluvia_total',
                            ('Tiempo_precipitacion', 'sum'):'tiempo_precipitacion',
                            ('minutos_luz', 'mean'):'minutos_luz', 'Mes': 'mes'}, inplace = True)
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'renombramiento del dataframe dfclima_est 1 (estatal)')
    print(f"Error renombramiento: {e}")

In [508]:
try:
  if not dfclima.empty:
    #Quitamos columnas ('salida_sol', 'mean') y ('puesta_sol', 'mean') antiguas y se obtiene 
    # el dataframe final por municipio, ordenando también las columnas
    df_clima_est = dfclima_est.drop([('salida_sol', 'mean'), ('puesta_sol', 'mean')], axis = 1)
    df_clima_est = df_clima_est[['anio', 'mes', 'entidad', 'temperatura_maxima',
                                'temperatura_minima', 'temperatura_promedio', 'salida_sol',
                                'puesta_sol', 'precipitacion_total', 'lluvia_total', 'tiempo_precipitacion', 'minutos_luz']]
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza dfclima_est 3 (estatal)')
    print(f"Error renombramiento: {e}")

In [509]:
try:
  if not dfclima.empty:
    # Redondeamos con dos decimales las columnas mostradas
    df_clima_est[['temperatura_maxima', 'temperatura_minima']] = df_clima_est[['temperatura_maxima', 'temperatura_minima']].round(decimals = 2)
    df_clima_est[['temperatura_promedio', 'minutos_luz']] = df_clima_est[['temperatura_promedio', 'minutos_luz']].round(decimals = 2)
    df_clima_est[['precipitacion_total', 'lluvia_total']] = df_clima_est[['precipitacion_total', 'lluvia_total']].round(decimals = 2)
  else:
    log_error_write(f"DataFrame leido vacio", 'Lectura vacios de dfclima')
    print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error renombramiento: {e}", 'Limpieza dfclima_est 4 (estatal)')
    print(f"Error renombramiento: {e}")

In [510]:
if not dfclima.empty:
    print(df_clima_est.head(3))

   anio      mes entidad  temperatura_maxima  temperatura_minima  \
0  2015    Enero  Sonora               22.37                8.41   
1  2015  Febrero  Sonora               25.47               10.48   
2  2015    Marzo  Sonora               26.97               12.25   

   temperatura_promedio salida_sol puesta_sol  precipitacion_total  \
0                 15.02       7:13      17:43                267.3   
1                 17.70       6:58      18:07                108.7   
2                 19.81       6:27      18:28                153.1   

   lluvia_total  tiempo_precipitacion  minutos_luz  
0         267.3                 280.0       629.88  
1         108.7                 134.0       669.46  
2         153.1                 214.0       720.23  


#### Generacion de archivo que contiene los datos de la tabla tidy Meteorologia_Climatologia_Estatal

In [511]:
# Pasamos este dataframe a archivo csv
try:
  if not df_clima_est.empty:
    create_csv_from_DataFrame(df_clima_est, f'Meteorologia_Climatologia_Estatal.csv')
  else:
      log_error_write(f"DataFrame leido vacio", 'Lectura vacios de df_clima_est')
      print(f"DataFrame leido vacio")
except Exception as e:
    # Imprimiendo un mensaje de error en caso de excepción
    log_error_write(f"Error creacion de archivo csv: {e}", 'creacion archivo tabla tidy Meteorologia_Climatologia_Estatal')
    print(f"Error creacion de archivo csv: {e}")

DataFrame (tabla tidy) guardado exitosamente en ../tidydata\Meteorologia_Climatologia_Estatal.csv


In [512]:
# Creando el archivo de texto con descripción general de las fuentes de descargar y parte del proceso ETL

# Uniendo elementos de la lista de las fuentes en donde se han descargado los datos (sources_list)
texto_list = "\n".join(sources_list)

date_now = datetime.now()

# Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
description = f"""
Proyecto: contando una historia a través de los datos (análisis de la variabilidad del comportamiento delictivo
en relación con factores temporales y geográficos, y su correlación con el desarrollo social y urbano para la prevención del delito)

Equipo participante:
- Ernesto
- Luis Andrés
- Mario Estrada Ferreira

Descripción de los datos descargados:
- Las fuentes fueron descargadas y donde se puede encontrar información adicional en el sitio web: {incidencia_delictiva_link}

Fuentes:
{texto_list}

Fechas de descarga:
- El proceso de descarga de los datos se realizó el: {date_now.strftime('%Y-%m-%d %H:%M:%S')}
"""
# Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
with open(f'fuentes_descripcion{date_now.strftime("%d%m%y")}.txt', 'w') as description_file:
    description_file.write(description)

print("Archivos de datos descargados y descripción en txt creada.")

Archivos de datos descargados y descripción en txt creada.
